# Preparando el proceso de ingesta de datos.

En este notebook vamos a realizar el proceso de ingesta de datos nuevos para la base de datos y poder así poder tener información continua para realizar un modelado de datos.

En este punto vamos a proceder a trabajar con código que hemos ido desarrollando en los anteriores notebooks.

    - Obtención de datos de la página web, pero aplicado a un determinado tiempo, que será periódico.
    - Ingeniería básica para preparar los .csv necesarios  para la carga de datos en la BBDD.
    - Carga de datos nuevos en la BBDD.

Vamos a intentar reducir el codigo aplicado en relación con los notebooks anteriores. Para ello vamos a recurrir a funciones que generaremos a partir de los codigos anteriores y que estarán en el fichero de [**functions.py**](..\Utils\functions.py)

Es por ello que aquí se verá solo el código sencillo de llamamiento a funciones donde generará primero unos ficheros .csv para después cargar los mismos en la base de datos.

In [2]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
import os
import pickle
import re
from datetime import datetime
import datetime
import spacy
from nltk.stem.snowball import SnowballStemmer
from Utils import functions as f

In [48]:
'''Variables'''

url = "https://www.amantis.net/productos-amantis/"              # lista productos
url_principal="https://www.amantis.net/"                        # productos
pages= np.arange(1,5)

date=str(datetime.today().strftime('%y%m%d'))
folder_ingest=r'.\Data'

ext=r'.csv'
scrape='_scrape'
file_product=r'\productos'
file_user=r'\usuarios'
file_comment=r'\comentarios'
file_price=r'\precios'
file_tag=r'\tags'
file_ingest_product=file_product+scrape
file_ingest_comment=file_comment+scrape

'''Mapeo de meses'''
dm_mapping={
    'enero':1, 
    'febrero':2, 
    'marzo':3, 
    'abril':4, 
    'mayo':5,
    'junio':6, 
    'julio':7,
    'agosto':8, 
    'septiembre':9, 
    'octubre':10, 
    'noviembre':11, 
    'diciembre':12,
} 

'''Listas a generar con la información de los productos'''
lista_URLs = []
name=[]
regular_prices=[]
new_price=[]
info=[]
id=[]
comentarios=[]


'''Generamos 2 diccionarios con los datos importantes para ingresar en una BBDD'''

diccionario_datos_productos={"ID":id,"NAME":name,"INFO":info,"LISTA_URL":lista_URLs,"REGULAR_PRICE":regular_prices,"DISCOUNT_PRICE":new_price}

diccionario_comentarios_productos={"ID":id,"COMENTARIOS":comentarios}

'''Listas para generar la información de los comentarios'''
id_comment=[]
comments=[]
date=[]
ratio=[]
users=[]
comment=[]

nombre_listas=['amenities','anal','BDSM','femenino','masculino','juguetes','lenceria','muebles']



In [ ]:

''' Obtenemos las URLs de los productos para entrar luego en sus URLS y extraer la información'''

print("Empezando a recoger datos de las paginas")
for page in pages:
    
    if page == 1:
        URL = url
        response = requests.get(url)
        soup = bs(response.text, 'lxml')
        productos = soup.find_all(class_='caption')
        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
        
    else:
        URL = url+'page' + str(page)+'/'
        response = requests.get(URL)
        soup = bs(response.text, 'lxml')
        productos = soup.find_all(class_='caption')
        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
print("Terminando de recoger los datos de los links de las paginas\nEmpezando a generar las listas de los productos")

for i in range(len(lista_URLs)):
    id.append(i)

    
'''Extraemos la información de cada producto existente'''

for URL in lista_URLs:
    url_product=URL
    response_product = requests.get(url_product)
    soup_product = bs(response_product.text, 'lxml')
    user_comments_product=[]
    date_comments_product=[]
    comments_product=[]
    rating=[]

    titulos=soup_product.find_all("h1",class_="h3")
    for titulo in titulos:
        nombre=titulo.get_text(strip=True)
        name.append(nombre)

    all_price = soup_product.find_all("div", class_="productoPrecio pull-right tdd_precio")                        
    for price_container in all_price:                                                                    
        try:
            special_price = price_container.find("span", class_="productSpecialPrice")
            if special_price:
                item_price = float(special_price.get_text(strip=True).replace(",", ".").split('€')[0])
                new_price.append(item_price)
                regular_price = price_container.find("del").get_text(strip=True)
                item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                regular_prices.append(item_regular_price)
            else:
                regular_price = price_container.find("span").get_text(strip=True)
                item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                new_price.append(item_regular_price)
                regular_prices.append(None)
        except:
            new_price.append(None)
            regular_prices.append(None)

    description=soup_product.find("div", class_="description") 
    information=description.get_text().split('\n')[1:]
    documentation = ''.join(information)
    info.append(documentation)


    '''Vamos a obtener los datos de los comentarios de los usuarios'''
    print("Cargando los datos de los comentarios")

    all_user_comments = soup_product.find_all("span", class_="name-user") 
    for user_comment in all_user_comments:
        user_comments_product.append(user_comment.get_text(strip=True))

       
    all_dates = soup_product.find_all("span", class_="date")  
    for dates in all_dates:
        dates_text=dates.get_text(strip=True)
        # dates=datetime.strftime(dates, '%dd/%mm/%Y')
        date_comments_product.append(dates_text)
        # date_object = datetime.strptime(date_comments_product)

    all_comments = soup_product.find_all("p")
    for formats in all_comments[-len(date_comments_product):]:
        comments_product.append(formats.get_text(strip=True))

    hearts = soup_product.find_all('div', class_= 'box-description')
    for heart in hearts:
        heart_rating = heart.find_all('span', class_= 'fas fa-heart')
        num_hearts = len(heart_rating)
        rating.append(num_hearts)

    datos = list(zip(date_comments_product,rating, user_comments_product,comments_product ))
    comentarios.append(datos)

for i, regular_price in enumerate(regular_prices):
    if regular_price is None:
        regular_prices[i] = new_price[i]

print("Realizando la ingenieria de los datos\nEliminando duplicados de productos")
noduplicated_product = productos.drop_duplicates(subset='NAME', keep='first')
removed_id = productos[productos.duplicated(subset='NAME', keep='first')]['ID']

print("Tratando los comentarios")

comentarios_productos=pd.DataFrame(diccionario_comentarios_productos)
comentarios=pd.DataFrame()
diccionario={"id":id_comment,"comments":comments}

for id_product,n_comments in enumerate (comentarios_productos['COMENTARIOS']):
    for i in n_comments:
        id_comment.append(id_product)
        comments.append(i)


for j in range(len(diccionario['comments'])):
    date.append(diccionario['comments'][j][0])
    ratio.append(diccionario['comments'][j][1])
    users.append(diccionario['comments'][j][2])
    comment.append(diccionario['comments'][j][3])


comentarios['ID']=pd.Series(id_comment)
comentarios['DATE']=pd.Series(date)
comentarios['RATIO']=pd.Series(ratio)
comentarios['USERS']=pd.Series(users)
comentarios['COMMENT']=pd.Series(comment)
comentarios.info()

noduplicated_comments = comentarios[~comentarios['ID'].isin(productos[productos['ID'].isin(removed_id)]['ID'])]
noduplicated_comments.info()

noduplicated_product.to_csv(folder_ingest+file_ingest_product+'_'+date+ext,header=True,index=False)           # Tengo que generar el path correcto
noduplicated_comments.to_csv(folder_ingest+file_ingest_comment+'_'+date+ext,header=True,index=True)           # Tengo que generar el path correcto



In [ ]:
'''Generando nuevos dataframes'''
print("Generando el fichero de productos")
dataframe=pd.read_csv(folder_ingest+file_ingest_product+'_'+date+ext)
dataframe['NAME'] = dataframe['NAME'].str.replace(r'-(?=\w)', '_')
dataframe[['PRODUCT', 'SLOGAN']] = dataframe['NAME'].str.split('[,-.]', 1, expand=True)
dataframe['PRODUCT'] = dataframe['PRODUCT'].str.strip()
dataframe['SLOGAN'] = dataframe['SLOGAN'].str.strip()
dataframe['CHARACTERISTICS'] = dataframe['INFO'].str.split('Ver características y medidas|Características', 1).str[1]
dataframe['DESCRIPTION'] = dataframe['INFO'].str.split('Ver características y medidas|Características', 1).str[0].str.strip()
dataframe['CHARACTERISTICS'] = dataframe['CHARACTERISTICS'].str.replace('\r', ' ')
dataframe['DESCRIPTION'] = dataframe['DESCRIPTION'].str.replace('\r', ' ')

col_1 = dataframe.pop('PRODUCT')
col_2=dataframe.pop('SLOGAN')
col_3=dataframe.pop('DESCRIPTION')
col_4=dataframe.pop('CHARACTERISTICS')

dataframe.drop(columns=['NAME'],inplace=True)
dataframe.drop(columns=['INFO'],inplace=True)

dataframe.insert(loc= 1 , column= 'PRODUCT', value= col_1)
dataframe.insert(loc= 2 , column= 'SLOGAN', value= col_2)
dataframe.insert(loc= 3 , column= 'DESCRIPTION', value= col_3)
dataframe.insert(loc= 4 , column= 'CHARACTERISTICS', value= col_4)
df_product=dataframe.iloc[:,:6]

print("Generando el fichero de tags")
df_engineer=pd.read_csv(folder_ingest+file_product+'_'+date+ext)
df_engineer['SLOGAN'] = df_engineer['SLOGAN'].str.lower()
df_engineer['DESCRIPTION'] = df_engineer['DESCRIPTION'].str.lower()
# df_engineer['SLOGAN'] = df_engineer['SLOGAN'].apply(f.eliminacion_acentos)              # Esto da error, debe de ser porque hay NaN en el campo
df_engineer['DESCRIPTION'] = df_engineer['DESCRIPTION'].apply(f.eliminacion_acentos)
listas = f.cargar_listas_desde_pickles(nombre_listas)
for nombre_lista in listas:
    df_engineer = f.aplicar_funcion_a_columna(df_engineer, nombre_lista)

print("Generando el fichero de precios")
df_engineer.to_csv(folder_ingest+file_tag+'_'+date+ext,header=True,index=False)
df_product.to_csv(folder_ingest+file_product+'_'+date+ext,header=True,index=False)
dataframe=pd.read_csv(folder_ingest+file_ingest_product+'_'+date+ext)
col_1 = dataframe.pop('REGULAR_PRICE')
col_2=dataframe.pop('DISCOUNT_PRICE')
col_3=dataframe['ID']
dataframe.insert(loc= 1 , column= 'ID_PRODUCT', value= col_3)
dataframe.insert(loc= 2 , column= 'REGULAR_PRICE', value= col_1)
dataframe.insert(loc= 3 , column= 'DISCOUNT_PRICE', value= col_2)
date_price=datetime.today().strftime('%y/%m/%d')
df_prices=dataframe.iloc[:,:4]
df_prices['FECHA']=date_price
df_prices.to_csv(folder_ingest+file_price+'_'+date+ext,header=True,index=False)

print("Generando el fichero de comentarios\ny de usuarios")
df_comments=pd.read_csv(folder_ingest+file_ingest_comment+'_'+date+ext)
'''Tratando a las Fechas'''

df_comments['DAY']=df_comments['DATE'].str.split(' ').str.get(1).astype('Int64')
df_comments['MONTH']=df_comments['DATE'].str.split(' ').str.get(2).str.split(',').str.get(0)
df_comments['YEAR']=df_comments['DATE'].str.split(' ').str.get(-1).astype('Int64')
df_comments['MONTH']=df_comments['MONTH'].map(dm_mapping)
df_comments['DATE'] = pd.to_datetime(df_comments.iloc[:,-3:])
df_comments=df_comments.iloc[:,:-3]

'''Tratando a los Usuarios'''

nombre_count = {}
count = {}

for i, row in df_comments.iterrows():
    id = row['ID']
    nombre = row['USERS']
    
    if id not in count:
        count[id] = {}
        
    if nombre in count[id]:
        count[id][nombre] += 1
        nueva_nombre = f"{nombre}_{count[id][nombre]}"
        df_comments.loc[i, 'USERS'] = nueva_nombre
    else:
        count[id][nombre] = 1
lista_users=df_comments['USERS'].unique()
mivalor = [ x for x in range(len(lista_users))]             
lista_users=list(lista_users)                                
lista_users_code = {k: v for k, v in zip(lista_users, mivalor)}   
df_comments['ID_USERS']= df_comments['USERS'].map(lista_users_code)
df_users=pd.DataFrame()
df_users['ID_USERS']=df_comments['ID_USERS']
df_users['USERS']=df_comments['USERS']
df_users.drop_duplicates(subset='ID_USERS', keep='first',inplace=True)
col = df_comments.pop('ID_USERS')
df_comments.drop(columns=['USERS'],inplace=True)
df_comments.insert(loc= 4 , column= 'ID_USERS', value= col)

df_comments.to_csv(folder_ingest+file_comment+'_'+date+ext,header=True,index=False)           # Tengo que generar el path correcto
df_users.to_csv(folder_ingest+file_user+'_'+date+ext,header=True,index=False)

